# Przetwarzanie wstępne. Filtracja kontekstowa.


### Cel:
- zapoznanie z pojęciem kontekstu / filtracji kontekstowej,
- zapoznanie z pojęciem konwolucji (splotu),
- zapoznanie z wybranymi filtrami:
	- filtry liniowe dolnoprzepustowe:
		- filtr uśredniający,
		- filtr Gaussa.
	- filtry nielinowe:
		- mediana,
		- mediana dla obrazów kolorowych.
	- filtry liniowe górnoprzepustowe:
			- laplasjan,
			- operator Robertsa, Prewitta, Sobela.
- zadanie domowe: adaptacyjna filtracja medianowa.

### Filtry liniowe uśredniające (dolnoprzepustowe)

Jest to podstawowa rodzina filtrów stosowana w cyfrowym przetwarzaniu obrazów. 
Wykorzystuje się je w celu "rozmazania" obrazu i tym samym redukcji szumów (zakłóceń) na obrazie.
Filtr określony jest przez dwa parametry: rozmiar maski (ang. _kernel_) oraz wartości współczynników maski.

Warto zwrócić uwagę, że omawiane w niniejszym rozdziale operacje generują nową wartość piksela na podstawie pewnego fragmentu obrazu (tj. kontekstu), a nie jak operacje punktowe tylko na podstawie jednego piksela.


1. Wczytaj obraz _plansza.png_.
W dalszej części ćwiczenia sprawdzenie działania filtracji dla innych obrazów sprowadzi się do wczytania innego pliku.

2. Podstawowa funkcja to `cv2.filter2D`  - realizacja filtracji konwolucyjnej.
   Proszę sprawdzić jej dokumentację i zwrócić uwagę na obsługę problemu brzegowego (na krawędziach istnieją piksele dla których nie da się wyznaczyć otoczenia).

  Uwaga. Problem ten można też rozwiązać z użyciem funkcji `signal.convolve2d` z biblioteki _scipy_ (`from scipy import signal`).

3. Stwórz podstawowy filtr uśredniający o rozmiarze $3 \times 3$ -- za pomocą funkcji `np.ones`. Wykonaj konwolucję na wczytanym obrazie. Na wspólnym rysunku wyświetl obraz oryginalny, po filtracji oraz moduł z różnicy.

4. Przeanalizuj otrzymane wyniki. Jakie elementy zawiera obraz "moduł z różnicy"? Co na tej podstawie można powiedzieć o filtracji dolnoprzepustowej?

In [ ]:
import matplotlib.pyplot as plt
import cv2
import os
import numpy as np
from scipy import signal


if not os.path.exists("jet.png") :
    !wget https://raw.githubusercontent.com/vision-agh/poc_sw/master/06_Context/jet.png --no-check-certificate
if not os.path.exists("kw.png") :
    !wget https://raw.githubusercontent.com/vision-agh/poc_sw/master/06_Context/kw.png --no-check-certificate
if not os.path.exists("moon.png") :
    !wget https://raw.githubusercontent.com/vision-agh/poc_sw/master/06_Context/moon.png --no-check-certificate
if not os.path.exists("lenaSzum.png") :
    !wget https://raw.githubusercontent.com/vision-agh/poc_sw/master/06_Context/lenaSzum.png --no-check-certificate
if not os.path.exists("lena.png") :
    !wget https://raw.githubusercontent.com/vision-agh/poc_sw/master/06_Context/lena.png --no-check-certificate
if not os.path.exists("plansza.png") :
    !wget https://raw.githubusercontent.com/vision-agh/poc_sw/master/06_Context/plansza.png --no-check-certificate

In [ ]:
def show_3_images_in_subplot(image1, image2):
    fig, (ax1,ax2,ax3) = plt.subplots(1, 3, figsize = (16, 6))
    ax1.set_title("Original image")
    ax1.imshow(image1, 'gray')
    ax1.axis('off')

    ax2.set_title("Image after filtration")
    ax2.imshow(image2, 'gray')
    ax2.axis('off')

    diff_image = np.abs(image1 - image2)
    ax3.set_title("Absolute value from the difference between images")
    ax3.imshow(diff_image, 'gray')
    ax3.axis('off')
    plt.show()

In [ ]:
plansza = cv2.imread("plansza.png", cv2.IMREAD_GRAYSCALE)

filtered_image = cv2.filter2D(plansza, -1, np.ones((3, 3), np.float32)/(3*3))
filtered_image = cv2.normalize(filtered_image, None, 0, 255, cv2.NORM_MINMAX, dtype=cv2.CV_8U)

show_3_images_in_subplot(plansza, filtered_image)

5. Na wspólnym rysunku wyświetl wyniki filtracji uśredniającej z oknem o rozmiarze 3, 5, 9, 15 i 35. 
Wykorzystaj polecenie `plt.subplot`. 
Przeanalizuj wpływ rozmiaru maski na wynik. 

In [ ]:
def average_filtration(image, window):
    filtered_image = cv2.filter2D(image, -1, np.ones((window, window), np.float32)/(window*window))
    return filtered_image

f, (ax1, ax2, ax3, ax4, ax5) = plt.subplots(1, 5, figsize = (16, 6))
plansza_3 = average_filtration(plansza, 3)
ax1.imshow(plansza_3, 'gray')
ax1.set_title("Size window: 3")
ax1.axis('off')

plansza_5 = average_filtration(plansza, 5)
ax2.imshow(plansza_5, 'gray')
ax2.set_title("Size window: 5")
ax2.axis('off')

plansza_9 = average_filtration(plansza, 9)
ax3.imshow(plansza_9, 'gray')
ax3.set_title("Size window: 39")
ax3.axis('off')

plansza_15 = average_filtration(plansza, 15)
ax4.imshow(plansza_15, 'gray')
ax4.set_title("Size window: 15")
ax4.axis('off')

plansza_35 = average_filtration(plansza, 35)
ax5.imshow(plansza_35, 'gray')
ax5.set_title("Size window: 35")
ax5.axis('off')

6. Wczytaj obraz _lena.png_.
Zaobserwuj efekty filtracji dolnoprzepustowej dla obrazu rzeczywistego.

In [ ]:
lena = cv2.imread("lena.png", cv2.IMREAD_GRAYSCALE)

filtered_image = average_filtration(lena, 5)

show_3_images_in_subplot(lena, filtered_image)

7. Niekorzystny efekt towarzyszący wykonanym filtracjom dolnoprzepustowym to utrata ostrości. 
Częściowo można go zniwelować poprzez odpowiedni dobór maski. 
Wykorzystaj maskę:  `M = np.array([1 2 1; 2 4 2; 1 2 1])`. 
Przed obliczeniami należy jeszcze wykonać normalizację - podzielić każdy element maski przez sumę wszystkich elementów: `M = M/sum(sum(M));`.
Tak przygotowaną maskę wykorzystaj w konwolucji - wyświetl wyniki tak jak wcześniej.
Możliwe jest też wykorzystywanie innych masek - współczynniki można dopasowywać do konkretnego problemu.

In [ ]:
M = np.array([[1, 2, 1],
              [2, 4, 2],
              [1, 2, 1]])
M = M / sum(sum(M))

filtered_image = signal.convolve2d(lena, M, mode = 'same')

show_3_images_in_subplot(lena, filtered_image)

8. Skuteczną i często wykorzystywaną maską jest tzw. maska Gasussa.
Jest to zbiór liczb, które aproksymują dwuwymiarowy rozkład Gaussa. 
Parametrem jest odchylenie standardowe i rozmiar maski.

9. Wykorzystując przygotowaną funkcję `fgaussian` stwórz maskę o rozmiarze $5 \times 5$ i odchyleniu standardowym 0.5.
  Wykorzystując funkcję `mesh` zwizualizuj filtr.
  Sprawdź jak parametr `odchylenie standardowe` wpływa na `kształt` filtru.

  Uwaga. W OpenCV dostępna jest *dedykowana* funkcja do filtracji Gaussa - `GaussianBlur`.
  Proszę na jednym przykładzie porównać jej działanie z użytym wyżej rozwiązaniem.

10. Wykonaj filtrację dla wybranych (2--3) wartości odchylenia standardowego.


In [ ]:
def fgaussian(size, sigma):
     m = n = size
     h, k = m//2, n//2
     x, y = np.mgrid[-h:h+1, -k:k+1]
     g = np.exp(-(x**2 + y**2)/(2*sigma**2))
     return g /g.sum() 
    
    
def mesh(fun, size):
    fig = plt.figure()
    ax = fig.add_subplot(111, projection='3d')
    

    X = np.arange(-size//2, size//2, 1)
    Y = np.arange(-size//2, size//2, 1)
    X, Y = np.meshgrid(X, Y)
    Z = fun
    
    ax.plot_surface(X, Y, Z)
    
    plt.show()

In [ ]:
mask1 = fgaussian(5, 0.5)
mesh(mask1, 5)

In [ ]:
mask2 = fgaussian(5, 0.2)
mesh(mask2, 5)

In [ ]:
mask3 = fgaussian(5, 1.5)
mesh(mask3, 5)

In [ ]:
lena_gaussian = cv2.GaussianBlur(lena, (3, 3), 0.5)
show_3_images_in_subplot(lena,lena_gaussian)

In [ ]:
lena_gaussian = cv2.GaussianBlur(lena, (3, 3), 0.2)
show_3_images_in_subplot(lena,lena_gaussian)

In [ ]:
lena_gaussian = cv2.GaussianBlur(lena, (3, 3), 1.5)
show_3_images_in_subplot(lena, lena_gaussian)

### Filtry nieliniowe -- mediana

Filtry rozmywające redukują szum, ale niekorzystnie wpływają na ostrość obrazu.
Dlatego często wykorzystuje się filtry nieliniowe - np. filtr medianowy (dla przypomnienia: mediana - środkowa wartość w posortowanym ciągu liczb).

Podstawowa różnica pomiędzy filtrami liniowymi, a nieliniowymi polega na tym, że przy filtracji liniowej na nową wartość piksela ma wpływ wartość wszystkich pikseli z otoczenia (np. uśrednianie, czasem ważone), natomiast w przypadku filtracji nieliniowej jako nowy piksel wybierana jest któraś z wartości otoczenia - według jakiegoś wskaźnika (wartość największa, najmniejsza czy właśnie mediana).


1. Wczytaj obraz _lenaSzum.png_ (losowe 10% pikseli białych lub czarnych - tzw. zakłócenia impulsowe). Przeprowadź filtrację uśredniającą z rozmiarem maski 3x3. Wyświetl, podobnie jak wcześniej, oryginał, wynik filtracji i moduł z różnicy. Wykorzystując funkcję `cv2.medianBlur` wykonaj filtrację medianową _lenaSzum.png_ (z rozmiarem maski $3 \times 3$). Wyświetl, podobnie jak wcześniej, oryginał, wynik filtracji i moduł z różnicy. Która filtracja lepiej radzi sobie z tego typu szumem?

  Uwaga. Taki sam efekt da również użycie funkcji `signal.medfilt2d`.


In [ ]:
lena_szum = cv2.imread("lenaSzum.png", cv2.IMREAD_GRAYSCALE)
filtered_lena_szum = average_filtration(lena_szum, 3)
show_3_images_in_subplot(lena_szum, filtered_lena_szum)

lena_szum_median = cv2.medianBlur(lena_szum, 3)
show_3_images_in_subplot(lena_szum, lena_szum_median)

2. Przeprowadź filtrację uśredniającą, a następnie medianową obrazu _lena.png_.
   Wyniki porównaj - dla obu wyświetl: oryginał, wynik filtracji i moduł z różnicy.
   Szczególną uwagę zwróć na ostrość i krawędzie.
   W której filtracji krawędzie zostają lepiej zachowane?

In [ ]:
filtered_lena = average_filtration(lena, 3)
show_3_images_in_subplot(lena, filtered_lena)

lena_median = cv2.medianBlur(lena, 3)
show_3_images_in_subplot(lena, lena_median)

3. Ciekawy efekt można uzyskać wykonując filtrację medianową wielokrotnie. Określa się go mianem  posteryzacji.  W wyniku przetwarzania z obrazka usunięte zostają detale, a duże obszary uzyskują tą samą wartość jasności.  Wykonaj operację mediany $5 \times 5$ na obrazie _lena.png_ 10-krotnie. (wykorzystaj np. pętlę `for`).


Inne filtry nieliniowe:
- filtr modowy - moda (dominanta) zamiast mediany,
- filtr olimpijski - średnia z podzbioru otoczenia (bez wartości ekstremalnych),
- hybrydowy filtr medianowy - mediana obliczana osobno w różnych podzbiorach otoczenia (np. kształt `x`, `+`), a jako wynik brana jest mediana ze zbioru wartość elementu centralnego, mediana z `x` i mediana z `+`,
- filtr minimalny i maksymalny (będą omówione przy okazji operacji morfologicznych w dalszej części kursu).


Warto zdawać sobie sprawę, z szerokich możliwości dopasowywania rodzaju filtracji do konkretnego rozważanego problemu i rodzaju zaszumienia występującego na obrazie.

In [ ]:
for _ in range(10):
    lena_median = cv2.medianBlur(lena, 5)

show_3_images_in_subplot(lena, lena_median)


## Filtry liniowe górnoprzepustowe (wyostrzające, wykrywające krawędzie)

Zadaniem filtrów górnoprzepustowych jest wydobywanie z obrazu składników odpowiedzialnych za szybkie zmiany jasności - konturów, krawędzi, drobnych elementów tekstury.

### Laplasjan (wykorzystanie drugiej pochodnej obrazu)

1. Wczytaj obraz _moon.png_.

2. Wprowadź podstawową maskę laplasjanu:
\begin{equation}
M = 
\begin{bmatrix}
0 & 1& 0 \\ 1 & -4 & 1 \\ 0 & 1 & 0
\end{bmatrix}
\tag{1}
\end{equation}

3. Przed rozpoczęciem obliczeń należy dokonać normalizacji maski - dla rozmiaru $3 \times 3$ podzielić każdy element przez sumę wag dodatnich (ewentualnie sumę modułów wszystkich wag).
   Proszę zwrócić uwagę, że nie można tu zastosować takiej samej normalizacji, jak dla filtrów dolnoprzepustowych, gdyż skutkowałby to dzieleniem przez 0.

4. Wykonaj konwolucję obrazu z maską (`c2.filter2D`). Przed wyświetleniem, wynikowy obraz należy poddać normalizacji (występują ujemne wartości). Najczęściej wykonuje się jedną z dwóch operacji:
- skalowanie (np. poprzez dodanie 128 do każdego z pikseli),
- moduł (wartość bezwzględna).

Wykonaj obie normalizacje. 
Na wspólnym wykresie wyświetl obraz oryginalny oraz przefiltrowany po obu normalizacjach. 

In [ ]:
def filter_image(image, mask):
    filtered_image = cv2.filter2D(image, -1, mask)
    filtered_image1 = filtered_image + 128
    filtered_image2 = np.abs(filtered_image)

    fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize = (16, 6))
    ax1.set_title("Original image")
    ax1.imshow(image, 'gray')
    ax1.axis('off')

    ax2.set_title("Image after filtration and addition of 128")
    ax2.imshow(filtered_image1, 'gray')
    ax2.axis('off')

    ax3.set_title("Absolute value from the filtered image")
    ax3.imshow(filtered_image2, 'gray')
    ax3.axis('off')
    plt.show()
    return filtered_image, filtered_image1, filtered_image2

In [ ]:
moon = cv2.imread("moon.png", cv2.IMREAD_GRAYSCALE)
M = np.array([[0, 1, 0],
              [1, -4, 1],
              [0, 1, 0]]) / 8

filtered_image = filter_image(moon, M)

7. Efekt wyostrzenia uzyskuje się po odjęciu/dodaniu (zależy do maski) rezultatu filtracji laplasjanowej i oryginalnego obrazu. Wyświetl na jednym wykresie: obraz oryginalny, sumę oryginału i wyniku filtracji oraz różnicę (bezwzględną) oryginału i wyniku filtracji.
 Uwaga. Aby uniknąć artefaktów, należy obraz wejściowy przekonwertować do formatu ze znakiem.



In [ ]:
moon_int = moon.astype('int16')

fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize = (16, 6))
ax1.set_title("Original image")
ax1.imshow(moon_int, 'gray')
ax1.axis('off')

ax2.set_title("The sum of the original and filtered image")
ax2.imshow(moon_int + filtered_image[1], 'gray')
ax2.axis('off')

ax3.set_title("Absolute difference of the original and filtered image")
ax3.imshow(np.abs(moon_int - filtered_image[2]), 'gray')
ax3.axis('off')

### Gradienty (wykorzystanie pierwszej pochodnej obrazu)

1. Wczytaj obraz _kw.png_. Stwórz odpowiednie maski opisane w kolejnych punktach i dokonaj filtracji.
2. Wykorzystując gradient Robertsa przeprowadź detekcję krawędzi - poprzez wykonanie konwolucji obrazu z daną maską:
\begin{equation}
R1 = \begin{bmatrix} 0 & 0 & 0 \\ -1 & 0 & 0 \\ 0 & 1 & 0 \end{bmatrix}   
R2 = \begin{bmatrix} 0 & 0 & 0 \\ 0 & 0 & -1 \\ 0 & 1 & 0 \end{bmatrix}
\tag{2}
\end{equation}

Wykorzystaj stworzony wcześniej kod (przy laplasjanie) - dwie metody normalizacji oraz sposób wyświetlania.

3. Analogicznie przeprowadź detekcję krawędzi za pomocą gradientu Prewitta (pionowy i poziomy)
\begin{equation}
P1 = \begin{bmatrix} -1 & 0 & 1 \\ -1 & 0 & 1 \\ -1 & 0 & 1 \end{bmatrix}   
P2 = \begin{bmatrix} -1 & -1 & -1 \\ 0 & 0 & 0 \\ 1 & 1 & 1 \end{bmatrix}
\tag{3}
\end{equation}

4. Podobnie skonstruowany jest gradient Sobela (występuje osiem masek, zaprezentowane są dwie `prostopadłe`):
\begin{equation}
S1 = \begin{bmatrix} -1 & 0 & 1 \\ -2 & 0 & 2 \\ -1 & 0 & 1 \end{bmatrix}   
S2 = \begin{bmatrix} -1 & -2 & -1 \\ 0 & 0 & 0 \\ 1 & 2 & 1 \end{bmatrix}
\tag{4}
\end{equation}

Przeprowadź detekcję krawędzi za pomocą gradientu Sobela. 

In [ ]:
kw = cv2.imread("kw.png", cv2.IMREAD_GRAYSCALE)

In [ ]:
R1=np.array([[0, 0, 0],
             [-1, 0, 0],
             [0, 1, 0]]) / 2

gradient_Robertsa1 = filter_image(kw, R1)

In [ ]:
R2=np.array([[0, 0, 0],
             [0, 0, -1],
             [0, 1, 0]]) / 2

gradient_Robertsa2 = filter_image(kw, R2)

In [ ]:
P1=np.array([[-1, 0, 1],
             [-1, 0, 1],
             [-1, 0, 1]]) / 6

gradient_Prewitta1 = filter_image(kw, P1)

In [ ]:
P2=np.array([[-1, -1, -1],
             [0, 0, 0],
             [1, 1, 1]]) / 6

gradient_Prewitta2 = filter_image(kw, P2)

In [ ]:
S1=np.array([[-1, 0, 1],
             [-2, 0, 2],
             [-1, 0, 1]]) / 8

gradient_Sobela1 = filter_image(kw, S1)

In [ ]:
S2=np.array([[-1, -2, -1],
             [0, 0, 0],
             [1, 2, 1]]) / 8

gradient_Sobela2 = filter_image(kw, S2)

5. Na podstawie dwóch ortogonalnych masek np. Sobela można stworzyć tzw. filtr kombinowany - pierwiastek kwadratowy z sumy kwadratów gradientów:
\begin{equation}
OW = \sqrt{(O * S1)^2 + (O * S2)^2}
\tag{5}
\end{equation}
gdzie:  $OW$ - obraz wyjściowy, $O$ - obraz oryginalny (wejściowy), $S1,S2$ - maski Sobela, $*$ - operacja konwolucji.

Zaimplementuj filtr kombinowany.

Uwaga. Proszę zwrócić uwagę na konieczność zmiany formatu danych obrazu wejściowego - na typ znakiem



In [ ]:
kw = cv2.imread("kw.png", cv2.IMREAD_GRAYSCALE)
kw = kw.astype('int16')

In [ ]:
def show_2_images_in_subplot(image1, image2):
    fig, (ax1,ax2) = plt.subplots(1, 2, figsize = (20, 12))
    ax1.set_title("Original image", fontsize = 20)
    ax1.imshow(image1, 'gray')
    ax1.axis('off')

    ax2.set_title("Image after filtration", fontsize = 20)
    ax2.imshow(image2, 'gray')
    ax2.axis('off')
    plt.show()

In [ ]:
def combined_filter1(image):
    X1 = cv2.filter2D(image, -1, S1)
    X1 = np.power(X1, 2)

    X2 = cv2.filter2D(image, -1, S2)
    X2 = np.power(X2, 2)

    OW = np.sqrt(X1 + X2)
    return OW

In [ ]:
show_2_images_in_subplot(kw, combined_filter1(kw))

6. Istnieje alternatywna wersja filtra kombinowanego, która zamiast pierwiastka z sumy kwadratów wykorzystuje sumę modułów (prostsze obliczenia). 
Zaimplementuj tę wersję. 

In [ ]:
def combined_filter2(image):
    X1 = cv2.filter2D(image, -1, S1)
    X1 = np.abs(X1)

    X2 = cv2.filter2D(image, -1, S2)
    X2 = np.abs(X2)

    OW = (X1 + X2)
    return OW

In [ ]:
show_2_images_in_subplot(kw, combined_filter2(kw))

7. Wczytaj plik _jet.png_ (zamiast _kw.png_).
Sprawdź działanie obu wariantów filtracji kombinowanej.

In [ ]:
jet = cv2.imread("jet.png", cv2.IMREAD_GRAYSCALE)
jet=jet.astype('int16')

In [ ]:
show_2_images_in_subplot(jet, combined_filter1(jet))

In [ ]:
show_2_images_in_subplot(jet, combined_filter2(jet))